In [2]:
import ffmpeg

# Example: Converting an MP3 file to WAV
input_file = "/Users/lschoebitz/Documents/gitrepos/gh-larnsce/mywhisper/data/audio/meeting01/241118_1304.mp3"
output_file = "/Users/lschoebitz/Documents/gitrepos/gh-larnsce/mywhisper/data/audio/meeting01/241118_1304.wav"

try:
    stream = ffmpeg.input(input_file)
    stream = ffmpeg.output(stream, output_file)
    ffmpeg.run(stream)
    print("Conversion successful")
except ffmpeg.Error as e:
    print(f"Error: {e.stderr.decode()}")

Conversion successful


ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 14.0.6
  configuration: --prefix=/opt/anaconda3/envs/transcription --cc=arm64-apple-darwin20.0.0-clang --ar=arm64-apple-darwin20.0.0-ar --nm=arm64-apple-darwin20.0.0-nm --ranlib=arm64-apple-darwin20.0.0-ranlib --strip=arm64-apple-darwin20.0.0-strip --disable-doc --enable-swresample --enable-swscale --enable-openssl --enable-libxml2 --enable-libtheora --enable-demuxer=dash --enable-postproc --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libdav1d --enable-zlib --enable-libaom --enable-pic --enable-shared --disable-static --disable-gpl --enable-version3 --disable-sdl2 --enable-libopenh264 --enable-libopus --enable-libmp3lame --enable-libopenjpeg --enable-libvorbis --enable-pthreads --enable-libtesseract --enable-libvpx --enable-librsvg
  libavutil      58. 29.100 / 58. 29.100
  libavcodec     60. 31.102 / 60. 31.102
  libavformat    60. 16.100 

In [ ]:
import pandas as pd
import numpy as np
import pyannote.audio
print(pyannote.audio.__version__)

from pyannote.audio import Pipeline
import whisper

# Initialize Pyannote pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token="")

# Load audio file
audio_file = "/Users/lschoebitz/Documents/gitrepos/gh-larnsce/mywhisper/data/audio/meeting01/241118_1304.wav"

import torch

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Silicon GPU)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Send pipeline to the appropriate device
pipeline.to(device)

# Perform diarization
diarization = pipeline(audio_file)

# Load Whisper model
model = whisper.load_model("base.en")

# Transcribe audio
result = model.transcribe(audio_file)

# Initialize an empty dictionary to store the results
transcript = {}

transcribed_sections = result["segments"]

# Process diarization and transcription results
for segment, _, speaker in diarization.itertracks(yield_label=True):
    start = segment.start
    end = segment.end

    print(start)
    print(end)

    text = ""

    for section in transcribed_sections:
        if (section["start"] >= np.floor(start) and section["end"] <= np.ceil(end)):
            print(start,end,text)
            text += " " + section["text"]


    
    # Append the text to the corresponding speaker in the dictionary
    if speaker not in transcript:
        transcript[speaker] = []
    transcript[speaker].append((start, end, text))

# Convert the dictionary into a list of records for DataFrame creation
records = []
for speaker, segments in transcript.items():
    for start, end, text in segments:
        records.append({
            "Speaker": speaker,
            "Start": start,
            "End": end,
            "Text": text
        })

# Create a DataFrame from the records
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file
df.to_csv("transcription_results.csv", index=False)

print("Transcription results saved to 'transcription_results.csv'.")




3.3.2


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1. Bad things might happen unless you revert torch to 1.x.


INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
/opt/anaconda3/envs/transcription/lib/python3.10/site-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will exec

Using MPS (Apple Silicon GPU)


INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
/opt/anaconda3/envs/transcription/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value)

0.63846875
8.02971875
0.63846875 8.02971875 
0.63846875 8.02971875   So welcome to our interview today.
0.63846875 8.02971875   So welcome to our interview today.  I'm here being the interviewer,
8.02971875
8.04659375
9.059093750000002
14.695343750000003
16.24784375
19.67346875
20.41596875
25.714718750000003
20.41596875 25.714718750000003 
26.94659375
28.718468750000003
30.06846875
31.82346875
32.88659375
34.979093750000004
Transcription results saved to 'transcription_results.csv'.
